# Tutorial: converting existing code to serverless without internal code modifications

In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

from quantum_serverless import QuantumServerless, remote, get, put

---

There are couple of ways to make your code running as serverless code:
- wrapping entire functions / classes as actors or functions
- monkey-patching existing class functions to swap some parts of code

### Approach #1: wrapping function / classes as tasks / actors

We will work with qiskit and try to wrap some functions / classes as tasks / actors

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers import Backend
from qiskit.circuit.random import random_circuit
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeVigo, FakeAlmaden, FakeBrooklyn, FakeCasablanca

circuit = random_circuit(5, 3)
backend = AerSimulator.from_backend(FakeAlmaden())

circuit.draw()

┌───┐                   ┌───┐                            
q_0: ────┤ I ├───────────■───────┤ Z ├────────────────────────────
     ┌───┴───┴────┐      │       ├───┤           ┌───┐            
q_1: ┤ Rz(6.0996) ├──────┼───────┤ X ├───────────┤ H ├────────────
     └─────┬──────┘      │       └─┬─┘           └───┘            
q_2: ──────■─────────────■─────────┼───────────────■──────────────
                   ┌────────────┐  │             ┌─┴─┐            
q_3: ──────■───────┤ Ry(4.6963) ├──┼─────────────┤ Y ├────────────
         ┌─┴─┐     └────────────┘  │  ┌──────────┴───┴───────────┐
q_4: ────┤ H ├─────────────────────■──┤ U3(3.3532,0.0282,6.0889) ├
         └───┘                        └──────────────────────────┘

We have transpile function. Let's start with transpilation function.

You can call `transpile(circuits, backend)` to compile circuits to specific backend locally. 

In [3]:
# let's transpile function to see what it does
transpiled_circuit = transpile(circuit, backend)

transpiled_circuit.draw(idle_wires=False, fold=-1)

global phase: 0
                   ┌──────────────┐    ┌───┐    ┌───────────────┐    ┌───┐           ┌───┐                                                            
      q_4 -> 7 ────┤ U2(π/4,-π/2) ├────┤ X ├────┤ U2(-π/2,3π/4) ├────┤ X ├─────■─────┤ X ├────────────────────────────────────────────────────────────
                   └──────────────┘    └─┬─┘    └───────────────┘    └─┬─┘   ┌─┴─┐   └─┬─┘               ┌───────────────────────────┐                
ancilla_7 -> 8 ──────────────────────────┼─────────────────────────────■─────┤ X ├─────■──────────────■──┤ U3(2.9299,-3.1134,2.9474) ├────────────────
                                         │                                   └───┘        ┌───┐     ┌─┴─┐└────────┬─────────┬────────┘                
      q_0 -> 9 ──────────────────────────┼─────────────────────────────────────────────■──┤ X ├──■──┤ X ├─────────┤ U2(0,π) ├─────────────────────────
                                         │    ┌───────────────────┐                    │  └─┬─┘  │  └───┘         └─────────┘         ┌───┐┌─────────┐
     q_3 -> 12 ──────────────────────────■────┤ U3(1.5869,π/2,-π) ├────────────────────┼────┼────┼────────────────────────────────────┤ X ├┤ U1(π/2) ├
                                              └───────────────────┘       ┌─────────┐  │    │    │  ┌───┐         ┌─────────┐         └─┬─┘└─────────┘
     q_2 -> 13 ──────────────────────────■─────────────────────────────■──┤ U2(0,π) ├──┼────┼────┼──┤ X ├─────────┤ U2(0,π) ├───────────■─────────────
               ┌──────────────────────┐┌─┴─┐┌───────────────────────┐┌─┴─┐└─────────┘┌─┴─┐  │  ┌─┴─┐└─┬─┘         └┬───────┬┘                         
     q_1 -> 14 ┤ U1(3.04979548019677) ├┤ X ├┤ U1(-3.04979548019677) ├┤ X ├───────────┤ X ├──■──┤ X ├──■────────────┤ U1(π) ├──────────────────────────
               └──────────────────────┘└───┘└───────────────────────┘└───┘           └───┘     └───┘               └───────┘

We can turn this function in ray remote function, that can be executed in parallel on configured machine / cluster

In [4]:
remote_transpile = remote(transpile)

Now we have remote transpile function, we can try it out. But before let's create serverless class which will gives us executon context.

In [6]:
serverless = QuantumServerless()
serverless

QuantumServerless:
 | selected cluster: <Cluster: Local (all cores)>

In [7]:
with serverless.context():
    # execute remote transpile function and get back pointer to remote task, 
    #    so we can fetch results out of it
    task = remote_transpile.remote(circuit, backend)
    print(f"Pointer to task: {task}")
    
    # get actual results from task, 
    #    which will be our transpiled circuit
    remotely_transpiled_circuits = get(task)

remotely_transpiled_circuits.draw(idle_wires=False, fold=-1)

Pointer to task: ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)


global phase: 0
                   ┌──────────────┐    ┌───┐    ┌───────────────┐    ┌───┐           ┌───┐                                                            
      q_4 -> 7 ────┤ U2(π/4,-π/2) ├────┤ X ├────┤ U2(-π/2,3π/4) ├────┤ X ├─────■─────┤ X ├────────────────────────────────────────────────────────────
                   └──────────────┘    └─┬─┘    └───────────────┘    └─┬─┘   ┌─┴─┐   └─┬─┘               ┌───────────────────────────┐                
ancilla_7 -> 8 ──────────────────────────┼─────────────────────────────■─────┤ X ├─────■──────────────■──┤ U3(2.9299,-3.1134,2.9474) ├────────────────
                                         │                                   └───┘        ┌───┐     ┌─┴─┐└────────┬─────────┬────────┘                
      q_0 -> 9 ──────────────────────────┼─────────────────────────────────────────────■──┤ X ├──■──┤ X ├─────────┤ U2(0,π) ├─────────────────────────
                                         │    ┌───────────────────┐                    │  └─┬─┘  │  └───┘         └─────────┘         ┌───┐┌─────────┐
     q_3 -> 12 ──────────────────────────■────┤ U3(1.5869,π/2,-π) ├────────────────────┼────┼────┼────────────────────────────────────┤ X ├┤ U1(π/2) ├
                                              └───────────────────┘       ┌─────────┐  │    │    │  ┌───┐         ┌─────────┐         └─┬─┘└─────────┘
     q_2 -> 13 ──────────────────────────■─────────────────────────────■──┤ U2(0,π) ├──┼────┼────┼──┤ X ├─────────┤ U2(0,π) ├───────────■─────────────
               ┌──────────────────────┐┌─┴─┐┌───────────────────────┐┌─┴─┐└─────────┘┌─┴─┐  │  ┌─┴─┐└─┬─┘         └┬───────┬┘                         
     q_1 -> 14 ┤ U1(3.04979548019677) ├┤ X ├┤ U1(-3.04979548019677) ├┤ X ├───────────┤ X ├──■──┤ X ├──■────────────┤ U1(π) ├──────────────────────────
               └──────────────────────┘└───┘└───────────────────────┘└───┘           └───┘     └───┘               └───────┘

Because we have this function as ray remote function, we can run multiple of them in parallel

In [8]:
with serverless.context():
    # let's run 5 circuits transpilations in parallel
    tasks = [
        remote_transpile.remote(random_circuit(5, i + 1), backend)
        for i in range(5)
    ]

    # get all results when ready
    transpiled_circuits = get(tasks)
    
# look at our final transpiled circuit
transpiled_circuits[-1].draw(idle_wires=False, fold=-1)

global phase: 4.0125
                                                                                                                                                                                                                                          ┌───┐                             ┌───┐┌──────────┐     ┌───┐                          ┌───┐     ┌───┐┌──────────┐     ┌───┐     ┌───┐                    ┌─────────┐           ┌─────────┐┌───┐┌──────────┐┌───┐┌─────────┐┌───┐┌──────────┐┌───┐     ┌───┐                         ┌─────────┐ ┌───┐     ┌───┐┌───────────────┐     ┌───┐                                      ┌───┐     ┌─────────────┐                 
ancilla_7 -> 11 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────┤ X ├──────────────────────────■──┤ X ├┤ U1(-π/4) ├──■──┤ X ├──■────────────────────■──┤ X ├──■──┤ X ├┤ U1(-π/4) ├─────┤ X ├──■──┤ X ├──■─────────■───────┤ U1(π/4) ├───■────■──┤ U2(0,π) ├┤ X ├┤ U1(-π/4) ├┤ X ├┤ U1(π/4) ├┤ X ├┤ U1(-π/4) ├┤ X ├──■──┤ X ├──■───────────────────■──┤ U1(π/4) ├─┤ X ├──■──┤ X ├┤ U3(π,π/2,π/2) ├──■──┤ X ├──■────■──────────────────────────────┤ X ├──■──┤ U2(0,-3π/4) ├─────────────────
                                                                                                                              ┌──────────────────────┐                                                                │                   └─┬─┘                          │  └─┬─┘└──────────┘  │  └─┬─┘  │                    │  └─┬─┘  │  └─┬─┘├─────────┬┘     └─┬─┘  │  └─┬─┘  │       ┌─┴─┐     ├─────────┴┐┌─┴─┐  │  └─────────┘└─┬─┘└──────────┘└─┬─┘└─────────┘└─┬─┘└──────────┘└─┬─┘┌─┴─┐└─┬─┘┌─┴─┐┌─────────────┐  │  └─────────┘ └─┬─┘  │  └─┬─┘└───────────────┘┌─┴─┐└─┬─┘┌─┴─┐  │                              └─┬─┘  │  └─────────────┘                 
      q_0 -> 12 ──────────────────────────────────────────────────────────────────────────■────────────────────────────────■──┤ U1(2.27914099623462) ├────────────────────────────────────────────────────────────────┼─────────────────────┼────────────────────────────┼────■────────────────┼────┼────┼────────────────────┼────┼────┼────■──┤ U1(π/4) ├────────┼────┼────┼────┼───────┤ X ├─────┤ U1(-π/4) ├┤ X ├──┼───────────────┼────────────────■───────────────┼────────────────■──┤ X ├──■──┤ X ├┤ U2(0,-3π/4) ├──┼────────────────┼────┼────■───────────────────┤ X ├──■──┤ X ├──┼────────────────────────────────┼────┼──────────────────────────────────
                                                  ┌───┐                                 ┌─┴─┐   ┌───────────────────────┐┌─┴─┐├──────────────────────┤                                                                │                     │               ┌─────────┐  │                     │    │    │                    │    │    │       └─────────┘        │    │    │    │       └───┘     └──────────┘└───┘  │               │                                │                   └───┘     └───┘└─────────────┘  │                │    │                        └───┘     └───┘  │                                │    │                                  
      q_1 -> 13 ──────────────────────────■───────┤ X ├───────────────────────────■─────┤ X ├───┤ U1(0.315170003273544) ├┤ X ├┤ U1(3.01995153642231) ├─────────────────────────────────■──────────────────────────────┼────■────────────────┼───────────────┤ U2(0,0) ├──┼─────────────────────┼────┼────┼────────────────────┼────┼────┼──────────────────────────┼────┼────┼────┼────────────────────────────────────┼───────────────┼────────────────────────────────┼───────────────────────────────────────────────────┼────────────────┼────┼─────────────────────────────────────────┼────────────────────────────────┼────┼──────────────────────────────────
                                          │       └─┬─┘                           │     └───┘   └──────

---

Let's look at classes and how we can turn them into actors and run remotely.

We will look at simulator and run it as remote actor and try to use it methods to run circuits.

In [10]:
AerSimulatorActor = remote(AerSimulator)

In [11]:
with serverless.context():
    # create actor and launch it remotely
    actor = AerSimulatorActor.remote()
    
    # call actors function
    #    creating another actor from fake backend
    actors_task = actor.from_backend.remote(FakeAlmaden())
    
    # get results of out actors mathod run
    result = get(actors_task)
    
result

AerSimulator('aer_simulator(fake_almaden)'
             noise_model=<NoiseModel on ['u3', 'cx', 'u2', 'id', 'measure']>)

### Approach #2: Monkey-patching

Python is allowing you to change definitions of classes and function in a runtime.
This is useful if you need to patch a small chunk of class with your implementation.
In our case we will want to swap chunks of code that can be run in parallel with our ray calls.

Let's create a dummy class that we will be monkey-patching later

In [12]:
class DummyClass:
    def sum_n_times(self, a: int, b: int, n_times: int):
        """Do something n times."""
        
        result = []
        for i in range(n_times):
            result.append(a + b)
        return result     

---
Let's write patch function

In [13]:
@remote
def sum_remote(a: int, b: int):
    return a + b

def sum_n_times_patch(self, a: int, b: int, n_times: int):
    return get([
        sum_remote.remote(a, b)
        for _ in range(n_times)
    ])  

Patch it now

In [14]:
DummyClass.sum_n_times = sum_n_times_patch

Now we need to create instance of our patched class and run it

In [15]:
dummy_class = DummyClass()

with serverless.context():
    result = dummy_class.sum_n_times(1, 1, n_times=10)
    print(result)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


Function is leveraging parallelization internally now